## 1. Web Access

### 1. Modules & Variables

In [1]:
! pip install undetected_chromedriver

# 원격 조종 시에도 구글 로그인 가능
# 참고 링크
# https://github.com/ultrafunkamsterdam/undetected-chromedriver

In [2]:
import time
from datetime import datetime

from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC  # noqa

import undetected_chromedriver as uc 

from bs4 import BeautifulSoup

In [8]:
dict_gAccounts = {
    'kjaehw0207' : 'flapaqj1!%',
    'woghks.study' : 'ans_entrance00'
}

account_1 = list(dict_gAccounts.keys())[0]
password_1 = dict_gAccounts[account_1]

account_2 = list(dict_gAccounts.keys())[1]
password_2 = dict_gAccounts[account_2]

### 2. Web Access

In [3]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get('https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')

In [5]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
btn_sign_in = driver.find_element(By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')
btn_sign_in.click()

In [9]:
# 계정 이메일 입력
input_account = driver.find_element(By.XPATH, '//*[@id="identifierId"]')
input_account.send_keys(account_2)
time.sleep(1)

In [10]:
# [다음] 버튼 클릭해 패스워드 입력
btn_next_password = driver.find_element(By.XPATH, '//*[@id="identifierNext"]/div/button/span')
btn_next_password.click()
time.sleep(1)

In [35]:
driver.back()

'''
비밀번호를 입력했는데도 모래시계만 돌고 있을 때
driver.back() 명령어 진행하면 맞춤기록 초기 페이지로 이동
로그인 버튼부터 다시
'''

In [11]:
# 계정 비밀번호 입력
input_password = driver.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
input_password.send_keys(password_2)

In [12]:
# [다음] 버튼 입력해 로그인 클릭
btn_next_sign_in = driver.find_element(By.XPATH, '//*[@id="passwordNext"]/div/button')
btn_next_sign_in.click()

In [11]:
# 연락처 확인을 통한 개인 맞춤 설정 페이지
try: 
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')
     
    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')

맞춤 페이지 없음. 시청기록 수집을 시작합니다.


### 3. Data Collection

In [13]:
def get_day(weekday=datetime.today().weekday()):
    from datetime import datetime

    list = ['월', '화', '수', '목', '금', '토', '일']
    return list[weekday]

get_day()

'토'

In [14]:
# 항목별 Class 이름

nm_jsn_day = "i6CNtf"

nm_class_container = "xDtZAf" # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb" # 영상 제목 / tag = <a>
nm_class_adfilter = "SiEggd" # 채널표기 유무: 없는 경우 광고 / tag = <div> 
nm_class_RunningTime = "bI9urf" # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW" # 시청 시간 / tag = <div>

date_today = datetime.today().strftime("%Y-%m-%d")
day_today = get_day()

In [28]:
print(date_today)
print(day_today)

2023-02-04
토


In [56]:
contents = soup.find_all('c-wiz', class_='xDtZAf')
contents[1]['data-date']
contents[1].find('a', class_=nm_class_title).text


'Crimson Raptor Champion Spotlight | Parody - League of Legends'

In [61]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

contents = soup.find_all('c-wiz', class_='xDtZAf')
for idx, content in enumerate(contents):
    if content.find('div', 'SiEggd'):
        if content.find('div', class_=nm_class_RunningTime):
            print(f"{idx}: Watch Record")
            print(f"Watch Date: {content['data-date']}")
            print(f"Title: {content.find('a', class_=nm_class_title).text}")
            print(f"Running Time: {content.find('div', class_=nm_class_RunningTime).text}")
            print(f"Watch Time: {' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])}")
        else:
            print(f"{idx}: Community Post")
        
    else:
        print(f'{idx}: Ad or Search')
        
    print(' ')

0: Ad or Search
 
1: Watch Record
Watch Date: 20230204
Title: Crimson Raptor Champion Spotlight | Parody - League of Legends
Running Time: 9:45
Watch Time: 오전 1:58
 
2: Ad or Search
 
3: Watch Record
Watch Date: 20230204
Title: 젠지 vs. 리브 샌박 | 매치26 하이라이트 | 02.03 | 2023 LCK 스프링 스플릿
Running Time: 45:49
Watch Time: 오전 12:54
 
4: Watch Record
Watch Date: 20230204
Title: KT vs. 광동 | 매치25 하이라이트 | 02.03 | 2023 LCK 스프링 스플릿
Running Time: 26:07
Watch Time: 오전 12:53
 
5: Ad or Search
 
6: Ad or Search
 
7: Watch Record
Watch Date: 20230203
Title: 솔직히, 연봉 얼마 받으세요? 직무, 연차별 현실 연봉 | ①광화문
Running Time: 4:48
Watch Time: 오전 11:46
 
8: Watch Record
Watch Date: 20230203
Title: 갤S23 출시됐지만…값비싼 신제품 대신 중고 인기 / KBS  2023.02.03.
Running Time: 2:12
Watch Time: 오전 11:43
 
9: Ad or Search
 
10: Watch Record
Watch Date: 20230203
Title: 이직을 성공할 수 밖에 없는 방법
Running Time: 10:57
Watch Time: 오전 11:36
 
11: Ad or Search
 
12: Watch Record
Watch Date: 20230203
Title: [퍼센트] 명품 소비 자극한 '불평등'…MZ 69.5% "계층 이동 힘들 것" / JTBC 뉴스룸
Ru

In [ ]:
# ERROR - 커뮤니티 글 확인 기록
# 채널은 있는데, 영상이 없음
"""
80: NO ads
지금 이 순간 대생성AI 버블 시대가 열렸습니다. 닷컴버블, 스마트폰 혁명, 블록체인 열풍을 놓치신 개발자분들 이번 기회는 꼭 잡으시길 바랍니다!
🎬영상보기: https://youtu.be/4xU0mGr4g...
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[33], line 9
      7     print(f'{idx}: NO ads')
      8     print(content.find('a', class_=nm_class_title).text)
----> 9     print(content.find('div', class_=nm_class_RunningTime).text)
     10     print(' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2]))
     12 else:

AttributeError: 'NoneType' object has no attribute 'text'
"""

Day 마다 jsname = i6CNtf 로 구분됨
각 Day_jsname 아래는 c-wiz로 싸인 정보 태그가 존재

나는 Day - item - item - item - Day 구조를 아는데
컴퓨터는 그냥 문자열 연속임

find 함수로 찾을 수는 있는데
Day는 Day끼리, item은 item끼리 따로 찾아서
날짜 구분을 하기 어려워짐

태그를 잘 살펴보니
영상 별 리스트 담긴 c-wiz에도 data-date가 있음

오늘-어제 따라 구분할 것 없이 c-wiz 태그에서 추출하면 됨
c-wiz의 data-date를 토대로 요일 알아내는 것 필요


In [45]:
len(list.find_all('c-wiz'))


200

In [23]:
sample = containers[1]


Crimson Raptor Champion Spotlight | Parody - League of Legends
9:45
오전 1:58


In [48]:
list

<div jsname="bN97Pc" role="list"><div class="CW0isc KpksOc" data-date="20230204" data-timestamp="1.67543966932984e12" jscontroller="IcmULc" jslog="64845; track:impression" jsname="i6CNtf"><div class="YkIxob"><div class="MCZgpb"><h2 class="rp10kf">오늘</h2><div class="iM6vT" jsaction="JIbuQc:FhRcwc;"><button aria-label="오늘부터의 활동 모두 삭제" class="VfPpkd-Bz112c-LgbsSe yHy1rc eT1oJ mN1ivc" data-date="20230204" data-disable-idom="true" data-timestamp="1.67543966932984e12" jsaction="click:cOuCgd; mousedown:UX7yZ; mouseup:lbsD7e; mouseenter:tfO1Yc; mouseleave:JywGue; touchstart:p6p2H; touchmove:FwuNnf; touchend:yfqBxc; touchcancel:JMtRjd; focus:AHmuwe; blur:O22p3e; contextmenu:mg9Pef;mlnRJb:fLiPzd;" jscontroller="soHxf" jslog="69590; 4:WzNd; track:JIbuQc" jsname="E0u0jd"><div class="VfPpkd-Bz112c-Jh9lGc" jsname="s3Eaab"></div><div class="VfPpkd-Bz112c-J1Ukfc-LhBDec"></div><svg class="DI7Mnf NMm5M" focusable="false" height="24" viewbox="0 0 24 24" width="24"><path d="M19 6.41L17.59 5 12 10.59 6.41 

In [26]:
# 날짜별로 나누려면?
days = soup.find_all('div', class_= "MCZgpb")
for day in days:
    print(day.text)

오늘
어제
2월 2일
2월 1일


### Spare codes

In [ ]:
btn_acccount_info = driver.find_element(By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/div[2]/div/a')

btn_sign_out = '' 
# html document, iframe 태그 안에 들어있어 확인이 어려움.
# 여러 계정을 같이 분석할 경우 driver 초기화 방식으로 진행하자.

In [72]:
driver.close()

아이디 / 비밀번호 입력 없이 로그인을 직접 한다면?
- 보안 문제
- 로그인 한 후 driver 초기화 문제

<div class="MCZgpb"><h2 class="rp10kf">오늘</h2><div class="iM6vT" jsaction="JIbuQc:FhRcwc;"><button class="VfPpkd-Bz112c-LgbsSe yHy1rc eT1oJ mN1ivc" jscontroller="soHxf" jsaction="click:cOuCgd; mousedown:UX7yZ; mouseup:lbsD7e; mouseenter:tfO1Yc; mouseleave:JywGue; touchstart:p6p2H; touchmove:FwuNnf; touchend:yfqBxc; touchcancel:JMtRjd; focus:AHmuwe; blur:O22p3e; contextmenu:mg9Pef;mlnRJb:fLiPzd;" jsname="E0u0jd" data-disable-idom="true" aria-label="오늘부터의 활동 모두 삭제" data-timestamp="1.67543966932984e12" data-date="20230204" jslog="69590; 4:WzNd; track:JIbuQc"><div jsname="s3Eaab" class="VfPpkd-Bz112c-Jh9lGc"></div><div class="VfPpkd-Bz112c-J1Ukfc-LhBDec"></div><svg width="24" height="24" viewBox="0 0 24 24" focusable="false" class="DI7Mnf NMm5M"><path d="M19 6.41L17.59 5 12 10.59 6.41 5 5 6.41 10.59 12 5 17.59 6.41 19 12 13.41 17.59 19 19 17.59 13.41 12 19 6.41z"></path></svg></button></div></div>